In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import numpy as np

C:\Users\vella\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
df1 = pd.read_csv(r"C:\Users\vella\Desktop\Praju\College\Projects\Final year\Data\Tesla\stock_yfinance_data.csv")
df1.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA


In [51]:
df1.rename(columns = {'Stock Name':'Stock_Name'}, inplace = True)

In [52]:
names = ['TSLA']
df1 = df1[df1.Stock_Name.isin(names)]

In [53]:
df1

,Date,Open,High,Low,Close,Adj Close,Volume,Stock_Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA
...,...,...,...,...,...,...,...,...
247,2022-09-23,283.089996,284.500000,272.820007,275.329987,275.329987,63748400,TSLA
248,2022-09-26,271.829987,284.089996,270.309998,276.010010,276.010010,58076900,TSLA
249,2022-09-27,283.839996,288.670013,277.510010,282.940002,282.940002,61925200,TSLA
250,2022-09-28,283.079987,289.000000,277.570007,287.809998,287.809998,54664800,TSLA


In [54]:
df2 = df1.iloc[:,[0,4]]
df2.head()

,Date,Close
0,2021-09-30,258.493347
1,2021-10-01,258.406677
2,2021-10-04,260.510010
3,2021-10-05,260.196655
4,2021-10-06,260.916656


In [55]:
#Converting dates as datetime format
df2.Date = pd.to_datetime(df2.Date)

C:\Users\vella\Anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [281]:
#Converting date column as index
df3 = df2.set_index("Date")

In [282]:
df3

,Close
Date,
2021-09-30,258.493347
2021-10-01,258.406677
2021-10-04,260.510010
2021-10-05,260.196655
2021-10-06,260.916656
...,...
2022-09-23,275.329987
2022-09-26,276.010010
2022-09-27,282.940002


In [152]:
#Putting the dataframe into a variable "train"
train = df3.iloc[:74]

In [153]:
#Model fitting for prediction
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

In [154]:
n_input = 1
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

In [155]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [156]:
model.fit_generator(generator,epochs=6)

Epoch 1/6


C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


13/13 [==============================] - 1s 3ms/step - loss: 0.3224
Epoch 2/6
13/13 [==============================] - 0s 3ms/step - loss: 0.2700
Epoch 3/6
13/13 [==============================] - 0s 3ms/step - loss: 0.2231
Epoch 4/6
13/13 [==============================] - 0s 3ms/step - loss: 0.1781
Epoch 5/6
13/13 [==============================] - 0s 3ms/step - loss: 0.1352
Epoch 6/6
13/13 [==============================] - 0s 3ms/step - loss: 0.0994


In [157]:
pred_list = []

In [158]:
batch = train[-n_input:].reshape((1, n_input, n_features))

In [159]:
for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

1/1 [==============================] - 0s 116ms/step


In [160]:
from pandas.tseries.offsets import DateOffset
add_dates = [df3.iloc[:74].index[-1] + DateOffset(days = x) for x in range(0,2) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df3.iloc[:74].columns)

In [161]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

In [162]:
#df_proj = pd.concat([df3,df_predict], axis=1)

In [163]:
df_predict

,Prediction
2022-01-14,311.386542


In [151]:
df3.iloc[70:76]

,Close
Date,
2022-01-10,352.706665
2022-01-11,354.799988
2022-01-12,368.739990
2022-01-13,343.853333
2022-01-14,349.869995
2022-01-18,343.503326


In [93]:
predicted = pd.DataFrame()

In [167]:
for i in range(0,(len(df3)-49)):
    #Putting the dataframe into a variable "train"
    #print(i)
    train = df3.iloc[i:(i+50)]
    #Model fitting for prediction
    scaler = MinMaxScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    
    n_input = 1
    n_features = 1
    generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
    
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dropout(0.15))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit_generator(generator,epochs=6)
    
    pred_list = []
    
    batch = train[-n_input:].reshape((1, n_input, n_features))
    
    for i in range(n_input):   
        pred_list.append(model.predict(batch)[0]) 
        batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
    
    from pandas.tseries.offsets import DateOffset
    add_dates = [df3.iloc[i:(i+50)].index[-1] + DateOffset(days = x) for x in range(0,2) ]
    future_dates = pd.DataFrame(index=add_dates[1:],columns=df3.iloc[i:(i+50)].columns)
    
    df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])
    
    predicted = predicted.append(df_predict, ignore_index=True)

Epoch 1/6


C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


9/9 [==============================] - 1s 3ms/step - loss: 0.3313
Epoch 2/6
9/9 [==============================] - 0s 3ms/step - loss: 0.2897
Epoch 3/6
9/9 [==============================] - 0s 3ms/step - loss: 0.2515
Epoch 4/6
9/9 [==============================] - 0s 3ms/step - loss: 0.2151
Epoch 5/6
9/9 [==============================] - 0s 3ms/step - loss: 0.1804
Epoch 6/6
1/1 [==============================] - 0s 198ms/step
Epoch 1/6


C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


9/9 [==============================] - 1s 3ms/step - loss: 0.3404
Epoch 2/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2945
Epoch 3/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2612
Epoch 4/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2320
Epoch 5/6
9/9 [==============================] - 0s 4ms/step - loss: 0.1982
Epoch 6/6
1/1 [==============================] - 0s 141ms/step
Epoch 1/6


...

9/9 [==============================] - 1s 4ms/step - loss: 0.4888
Epoch 2/6
9/9 [==============================] - 0s 4ms/step - loss: 0.4350
Epoch 3/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3784
Epoch 4/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3289
Epoch 5/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2728
Epoch 6/6
1/1 [==============================] - 0s 136ms/step
Epoch 1/6


C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


9/9 [==============================] - 1s 4ms/step - loss: 0.4952
Epoch 2/6
9/9 [==============================] - 0s 4ms/step - loss: 0.4336
Epoch 3/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3779
Epoch 4/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3325
Epoch 5/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2868
Epoch 6/6
1/1 [==============================] - 0s 136ms/step
Epoch 1/6


C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


9/9 [==============================] - 1s 4ms/step - loss: 0.4293
Epoch 2/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3780
Epoch 3/6
9/9 [==============================] - 0s 4ms/step - loss: 0.3262
Epoch 4/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2852
Epoch 5/6
9/9 [==============================] - 0s 4ms/step - loss: 0.2391
Epoch 6/6
1/1 [==============================] - 0s 137ms/step


In [236]:
predicted

,Prediction
0,295.363618
1,292.607625
2,288.009458
3,287.413031
4,289.423148
...,...
198,257.544437
199,255.258682
200,262.927881
201,264.539307


In [290]:
df4 = pd.DataFrame(np.nan, index=[list(range(50))], columns=['Prediction'])

In [291]:
df4

,Prediction
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [292]:
df4 = df4.append(predicted, ignore_index=True)

In [293]:
df4.drop(df4.tail(1).index,inplace=True)
df4

,Prediction
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
247,258.505497
248,257.544437
249,255.258682
250,262.927881


In [280]:
df5 = df3
df5

,Close,Prediction
Date,,
2021-09-30,258.493347,NaN
2021-10-01,258.406677,NaN
2021-10-04,260.510010,NaN
2021-10-05,260.196655,NaN
2021-10-06,260.916656,NaN
...,...,...
2022-09-23,275.329987,NaN
2022-09-26,276.010010,NaN
2022-09-27,282.940002,NaN


In [294]:
extracted_col = list(df4["Prediction"])

In [295]:
display(extracted_col)

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 295.3636175417386,
 292.6076252522607,
 288.00945838961525,
 287.4130307888695,
 289.4231481192619,
 287.18290270297257,
 292.9498409919106,
 292.56235897528677,
 293.6228042219354,
 300.0043139050913,
 307.41200108192425,
 318.8188791113671,
 322.14384251408774,
 313.54003822540653,
 322.1592096319109,
 319.46498922279943,
 335.8272521115723,
 327.60639126408205,
 326.6487033036101,
 325.13636204793147,
 323.197116043033,
 325.2629043056877,
 320.8386652102563,
 322.976393159468,
 321.6725234225123,
 328.15226619004534,
 323.53419376738657,
 321.44519711741486,
 320.2495208935834,
 315.7696784044783,
 315.53283314002147,
 313.6359189552022,
 315.61566918633434,
 292.3939310358155,
 295.116

In [296]:
df5["Prediction"] = extracted_col

In [297]:
df5.iloc[40:100]

,Close,Prediction
Date,,
2021-11-26,360.640015,NaN
2021-11-29,378.996674,NaN
2021-11-30,381.586670,NaN
2021-12-01,365.000000,NaN
2021-12-02,361.533325,NaN
2021-12-03,338.323334,NaN
2021-12-06,336.336670,NaN
2021-12-07,350.583344,NaN
2021-12-08,356.320007,NaN


In [298]:
df5 = df5.fillna(0)

In [300]:
df5.tail(60)

,Close,Prediction
Date,,
2022-07-07,244.543335,228.013170
2022-07-08,250.763336,227.620406
2022-07-11,234.343338,226.375721
2022-07-12,233.070007,226.017453
2022-07-13,237.039993,225.873667
2022-07-14,238.313339,225.408690
2022-07-15,240.066666,227.272417
2022-07-18,240.546661,227.712622
2022-07-19,245.529999,226.837510


In [225]:
df3

,Close,Prediction
Date,,
2021-09-30,258.493347,NaN
2021-10-01,258.406677,NaN
2021-10-04,260.510010,NaN
2021-10-05,260.196655,NaN
2021-10-06,260.916656,NaN
...,...,...
2022-09-23,275.329987,NaN
2022-09-26,276.010010,NaN
2022-09-27,282.940002,NaN


In [71]:
df = pd.read_csv(r"C:\Users\vella\Desktop\Praju\College\Projects\Final year\Data\Comb_Sent.csv")
df.head()

,Unnamed: 0,Date,compound_x,compound_y
0,0,2021-09-30,0.213078,0.0000
1,1,2021-10-01,0.224785,0.0000
2,2,2021-10-02,0.255568,0.0000
3,3,2021-10-03,0.257956,-0.0772
4,4,2021-10-04,0.106842,0.0000
